# Convolutionnel Neural Networks For Facial Expression Recognition 
****
Emotion detection from facial expression is one of the most active research fields, and plays a huge part in today’s technology. It can be implemented using machine learning algorithms, although these can’t provide  a hundred percent accurate solution since facial expression are not always the same and they depend on the person, the brightness, the position, and so on. This Notebook, presents an implementation of a deep learning algorithm for emotion detection using Convolutional Neural Network after some pre-processing steps to prepare our data uing OpenCV. Our choice of using CNN for this matter is based on the fact that this algorithm performs better than other solutions. Also, to conduct this experiment we have used a dataset which is a mix of other datasets like JAFFE and that was provided by Kaggle in the context of a competition.

****
This work was made by:
    * Nasr Abdelhamid  
                        abdelnasr7@gmail.com
    * Omar Harchich 
                        omar.harchich@gmail.com
Supervised by:
    * Professor Elhannani Assmaa.
    * Mrs Fatima Zahra Salmam.


*****
Our proposed method is divided to the following steps:
    1. Data preparation
    2. Image Processing
    3. Build a COnvolutionnel Neural Networks Model
    4. Test the model on another Dataset

# 1. Data Preparation

At first, we need to analyse our dataset in order to understand the features of images.
*****
Now we start by loading our csv file that contains both image filename and its corresponding emotion.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_csv = pd.read_csv('../input/dataset/dataset/data/legend.csv')

We display the loaded csv

In [ ]:
train_csv.head()

When checking the csv file, we noticed that some images have a corresponding emotion in lower case whereas others in upper case as shown in the execution below :

In [ ]:
train_csv.groupby('emotion').count()

That said, before moving forward, we need to normalize the emotions in the csv file. We will thus convert all emotions into lower case :

In [ ]:
train_csv['emotion'] = train_csv['emotion'].str.lower()
train_csv.groupby('emotion').count()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

train_csv['emotion'].value_counts().plot(kind='bar')

As we can see in the plot above, the contempt emotion doesn't have much records in the dataset. Therefore, it will not be as usefull as the others. That's why we will remove it by adding its images into the angry category : 

In [ ]:
train_csv.replace("contempt", "anger", inplace=True)
train_csv.groupby('emotion').count()

In [ ]:
train_csv['emotion'].value_counts().plot(kind='bar')

After normalizing the emotions, we need to convert them into numeric labels in order to use them during the training process.
For that we will use the map() function and store the result in a new variable called mapping_emotion.

In [ ]:
mapping_emotion = {'anger': 0, 'disgust': 1, 'fear': 2, 'happiness': 3, 'neutral': 6, 'sadness': 4, 'surprise': 5}
train_csv['label'] = train_csv['emotion'].map(mapping_emotion)

In [ ]:
train_csv.head()

# 2. Image processing

Now that we have our dataset normalized, we can proceed with the processing step. 
The training set is a collection of images where some are colored and others are converted to gray scale. The same is valid for the test set that also contains both colored and grayscaled images. Thus, we need to perform a grayscale conversion on the training set. 

Let's first have a look at the training set. we will import the necessary libraries and then display a sample image of the training dataset that contains only grayscaled images :

In [ ]:
import cv2 as cv
# Load image 
img_1 = cv.imread('../input/dataset/dataset/images/Aaron_Eckhart_0001.jpg')

In [ ]:
plt.imshow(img_1,cmap='gray')

Colored images contain some information that is considered as noise in the image processing work. That's why we need to convert them to gray scale.

Down below is the colored image after being converted to gary scale.

In [ ]:
img_colored = cv.imread('../input/dataset/dataset/test/Katrinakaif_32.jpg')
#plt.imshow(cv2.cvtColor(img_colored, cv2.COLOR_BGR2RGB))

gray_image = cv.cvtColor(img_colored, cv.COLOR_BGR2GRAY) 
#plt.imshow(gray_image,cmap='gray')

plt.subplot(1,2,1)
plt.imshow(cv.cvtColor(img_colored, cv.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(gray_image,cmap='gray')

Colors are not the only obstacle that we need to avoid, sometimes images contains other types of noise like the background information and so on. That's why we also need to perform cropping on images in order to keep only the regions of interest.

In our case, we want to have images of size (96,96). But before cropping them we need to make sure that we keep the face region in the cropped image. That's why we need to proceed by detecting the face first, and then crop the image.

In [ ]:
face_cascade = cv.CascadeClassifier('../input/haarcascade/haarcascade_frontalface_default.xml')
height, width = gray_image.shape[:2]
face = face_cascade.detectMultiScale(gray_image, 1.3, 1)
if isinstance(face, tuple):
    resized_image = cv.resize(gray_image, (96, 96))
        #cv.imwrite(trained+'/'+name,resized_image)
elif isinstance(face, np.ndarray):
    for (x,y,w,h) in face:
        if w * h < (height * width) / 3:
            resized_image = cv.resize(gray_image, (96, 96)) 
                #cv.imwrite(trained+'/'+name,resized_image)
        else:
            roi_gray = gray_image[y:y+h, x:x+w]
            resized_image = cv.resize(roi_gray, (96, 96))
                #cv.imwrite(trained+'/'+name, resized_image)
    #if not name in deleting:
    #data1.append(img_to_array(resized_image))
plt.imshow(resized_image,cmap='gray')

Now that we have done the necessary processing on the sample images, let's do the same to the rest of the dataset :

In [ ]:
import glob
import cv2 as cv
import os
from keras.preprocessing.image import img_to_array

In [ ]:
trained = 'trainedimages'
os.mkdir(trained)

In [ ]:
face_cascade = cv.CascadeClassifier('../input/haarcascade/haarcascade_frontalface_default.xml')
image_train = '../input/dataset/dataset/images'

In [ ]:
data = []
labels = []

In [ ]:
i = 0
for img in glob.glob(image_train+"/*.jpg"):
    image = cv.imread(img)
    name = img.split('/')[-1]
    
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # convert to greyscale
    height, width = image.shape[:2]
    faces = face_cascade.detectMultiScale(gray_image, 1.3, 1)
    if isinstance(faces, tuple):
        resized_image = cv.resize(gray_image, (96, 96))
        cv.imwrite(trained+'/'+name,resized_image)
    #print(faces)
    elif isinstance(faces, np.ndarray):
        for (x,y,w,h) in faces:
            if w * h < (height * width) / 3:
                resized_image = cv.resize(gray_image, (96, 96)) 
                cv.imwrite(trained+'/'+name,resized_image)
            else:
                
                #cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray_image[y:y+h, x:x+w]
                #print(len(roi_gray))
                resized_image = cv.resize(roi_gray, (96, 96))
                cv.imwrite(trained+'/'+name, resized_image)
    
    #print(label, type(label), name)
    data.append(img_to_array(resized_image))
    label = int(train_csv[ train_csv['image'] == name][['label']].values)    
    labels.append(label)
    

The following code displays the resulting images :

In [ ]:
i = 1
import glob
plt.figure(0, figsize=(12,6))
for img in glob.glob(trained+'/*.jpg'):
    img = cv.imread(img)
    plt.subplot(4,4,i)
    plt.imshow(img, cmap="gray")
    i = i + 1
    if i == 17:
        break

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))
print(data.shape, labels.shape)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# 3. Build a Convolutionnel Neural Networks Model

In this part, we will build and train our model.

Let's import the necessary libraries first :

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

With deep learning, or any machine learning for that matter, a common practice is to make a training and testing split where we create an 70/30 random split of the data.

In [ ]:
# partition the data into training and testing splits using 70% of
# the data for training and the remaining 30% for testing
from sklearn.model_selection import train_test_split
(trainX, valX, trainY, valY) = train_test_split(data,labels, test_size=0.3, random_state=42)

The neural network will receive as input a 48x48 grayscale image and then output the confidence of each expression.

The network architecture comprises 5 convolutional layers, 3 subsampling layers and one fully connected layer. The first layer of the CNN is a convolution layer, that applies a convolution kernel of 3 ×3 and outputs 64 images of 96 × 96 pixels. This layer is followed by a subsampling layer that uses max-pooling (with kernel size 3 ×3) to reduce the image to the third of its size. The second and third convolutional layers, which output 64 images of 16 ×16 pixels, followed by a sub-sampling layer with kernel size 2 ×2. The fourth and fifth convolutional layers, output 128 images of size 8 ×8 pixels, and uses max pooling with kernel 2 ×2. The outputs are given to a fully connected hidden layer that has 1024 neurons. Finally, the network has six or seven output nodes (one for each expression that outputs their confidence level) that are fully connected to the previous layer.

In [ ]:
def buildModel(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# CONV => RELU => POOL
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.25))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

For the training part, we will train our model for a hundred epochs :

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 50
INIT_LR = 1e-3
BS = 32

In [ ]:
model = buildModel(width=96, height=96,depth=1, classes=len(lb.classes_))

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

* EPOCHS:  The total number of epochs we will be training our network for (i.e., how many times our network “sees” each training example and learns patterns from it).

* INIT_LR:  The initial learning rate — a value of 1e-3 is the default value for the Adam optimizer, the optimizer we will be using to train the network.

* BS:  We will be passing batches of images into our network for training. There are multiple batches per epoch. The BS  value controls the batch size.


Next, let’s create our image data augmentation object:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(valX, valY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
print('[INFO] Done!')

Once our Keras CNN has finished training, we’ll need to save both the model and label binarizer as we’ll need to load them from disk when we test the network on images outside of our training/testing set:

In [ ]:
import pickle

In [ ]:
model.save('emotion.model')
f = open("lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

Finally, we can plot our training and loss accuracy

In [ ]:
plt.figure(figsize=(14,3))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(H.history['loss'], color='b', label='Training Loss')
plt.plot(H.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(H.history['acc'], color='b', label='Training Accuracy')
plt.plot(H.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from keras.models import load_model
model = load_model('../input/modeltrain/emotion.model')

# 4. Test the model on another Dataset

Now we are ready to make a prediction with a set of images. These should be processed with the same method implemented in training data.

In [ ]:
image_test = '../input/dataset/dataset/test'
os.mkdir('test_pretraitement')

In [ ]:
test_pretraitement = 'test_pretraitement'

In [ ]:
data_test = {}
labels_test = {}
i = 0
for img in glob.glob(image_test+"/*.jpg"):
    image = cv.imread(img)
    name = img.split('/')[-1]
    
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # convert to greyscale
    height, width = image.shape[:2]
    faces = face_cascade.detectMultiScale(gray_image, 1.3, 1)
    if isinstance(faces, tuple):
        resized_image = cv.resize(gray_image, (48, 48))
        cv.imwrite(test_pretraitement+'/'+name,resized_image)
    #print(faces)
    elif isinstance(faces, np.ndarray):
        for (x,y,w,h) in faces:
            if w * h < (height * width) / 3:
                resized_image = cv.resize(gray_image, (48, 48)) 
                cv.imwrite(test_pretraitement+'/'+name,resized_image)
            else:
                
                #cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray_image[y:y+h, x:x+w]
                #print(len(roi_gray))
                resized_image = cv.resize(roi_gray, (48, 48))
                cv.imwrite(test_pretraitement+'/'+name, resized_image)
    image = resized_image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    data_test[name] = image
    #data.append(img_to_array(resized_image))
    
    #print(label, type(label), name)
    

From there we can start implementing our script to classify images that are not part of our training model.

In [ ]:
data_predict = {}
for key,value in data_test.items():
    predict = model.predict(value)
    idx = np.argmax(predict)
    l= lb.classes_[idx]
    data_predict[key] = l

In [ ]:
final_data = pd.DataFrame(list(data_predict.items()),
                      columns=['Image','Emotion'])

In [ ]:
mapping_emotion = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 6:'neutral', 4:'sadness', 5:'surprise'}
final_data['Emotion'] = final_data['Emotion'].map(mapping_emotion)

In [ ]:
final_data.to_csv('submissions.csv')